# Word2Vec using Gensim

In [8]:
"""
isiZulu Word Similarity Benchmark Evaluator - FIXED VERSION
Adapted for isiZulu word embeddings with support for:
- Custom isiZulu word similarity datasets
- SimLex-999 and WordSim-353 (if translated to isiZulu)
- Precision, Recall, Accuracy, and F1 Score metrics
- Handling of isiZulu morphology and agglutination
"""

import os
import csv
import numpy as np
from scipy.stats import spearmanr, pearsonr
from gensim.models import Word2Vec


def confusion_matrix_np(y_true, y_pred):
    """
    Compute confusion matrix without sklearn.
    
    Returns:
        tn, fp, fn, tp
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    tp = np.sum((y_true == 1) & (y_pred == 1))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    
    return tn, fp, fn, tp


def accuracy_np(tp, tn, fp, fn):
    """Compute accuracy."""
    total = tp + tn + fp + fn
    if total == 0:
        return 0.0
    return (tp + tn) / total


def precision_np(tp, fp):
    """Compute precision."""
    if (tp + fp) == 0:
        return 0.0
    return tp / (tp + fp)


def recall_np(tp, fn):
    """Compute recall."""
    if (tp + fn) == 0:
        return 0.0
    return tp / (tp + fn)


def f1_np(precision, recall):
    """Compute F1 score."""
    if (precision + recall) == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)


class IsiZuluBenchmarkEvaluator:
    """
    Enhanced evaluator for isiZulu word embeddings.
    Includes both correlation metrics and classification metrics.
    
    Handles isiZulu-specific challenges:
    - Agglutinative morphology (prefixes, suffixes)
    - Multiple word forms (singular/plural, noun classes)
    - Code-switching with English
    """
    
    def __init__(self, model=None, cache_size=1000, handle_missing='skip'):
        """
        Initialize evaluator with an isiZulu word embedding model.
        
        Args:
            model: Word2Vec model or any model with:
                   - wv.similarity(word1, word2) method, or
                   - wv.get_vector(word) or wv[word] to get vectors
            cache_size: Size of similarity cache (default: 1000)
            handle_missing: How to handle missing words:
                           'skip' - Skip pairs with missing words (default)
                           'zero' - Assign similarity of 0
                           'mean' - Assign mean similarity
        """
        self.model = model
        self.cache_size = cache_size
        self.similarity_cache = {}
        self.handle_missing = handle_missing
        
    def get_similarity(self, word1, word2):
        """
        Get cosine similarity between two isiZulu words.
        Uses cache to avoid redundant computations.
        
        Handles isiZulu-specific cases:
        - Lowercasing for consistency
        - Optional prefix/suffix variations
        """
        # Normalize words (lowercase for isiZulu)
        word1 = word1.lower().strip()
        word2 = word2.lower().strip()
        
        # Check cache first
        cache_key = tuple(sorted([word1, word2]))
        if cache_key in self.similarity_cache:
            return self.similarity_cache[cache_key]
        
        try:
            # Try using model's similarity method
            if hasattr(self.model, 'wv'):
                if hasattr(self.model.wv, 'similarity'):
                    sim = self.model.wv.similarity(word1, word2)
                else:
                    # Compute manually from vectors
                    vec1 = self.model.wv[word1]
                    vec2 = self.model.wv[word2]
                    sim = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
            elif hasattr(self.model, 'similarity'):
                sim = self.model.similarity(word1, word2)
            else:
                # Try direct indexing
                vec1 = self.model[word1]
                vec2 = self.model[word2]
                sim = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
            
            # Cache the result
            if len(self.similarity_cache) < self.cache_size:
                self.similarity_cache[cache_key] = sim
            
            return sim
            
        except (KeyError, AttributeError) as e:
            return None
    
    def binarize_scores(self, scores, threshold='median'):
        """
        Convert continuous scores to binary labels.
        
        Args:
            scores: Array of continuous scores
            threshold: 'median', 'mean', or a specific numeric value
            
        Returns:
            Binary array (1 for similar, 0 for dissimilar), threshold value
        """
        scores = np.array(scores)
        
        if threshold == 'median':
            thresh_val = np.median(scores)
        elif threshold == 'mean':
            thresh_val = np.mean(scores)
        else:
            thresh_val = threshold
            
        return (scores >= thresh_val).astype(int), thresh_val
    
    def compute_classification_metrics(self, human_scores, model_scores, 
                                       threshold='median', verbose=True):
        """
        Compute precision, recall, accuracy, and F1 score for isiZulu embeddings.
        
        Treats word similarity as a binary classification problem:
        - Similar pairs (high scores) vs. Dissimilar pairs (low scores)
        
        Args:
            human_scores: Ground truth similarity scores
            model_scores: Model-predicted similarity scores
            threshold: How to binarize scores ('median', 'mean', or numeric value)
            verbose: Whether to print detailed results
            
        Returns:
            Dictionary with classification metrics
        """
        # Binarize scores
        y_true, human_thresh = self.binarize_scores(human_scores, threshold)
        y_pred, model_thresh = self.binarize_scores(model_scores, threshold)
        
        # Compute confusion matrix
        tn, fp, fn, tp = confusion_matrix_np(y_true, y_pred)
        
        # Compute metrics
        accuracy = accuracy_np(tp, tn, fp, fn)
        precision = precision_np(tp, fp)
        recall = recall_np(tp, fn)
        f1 = f1_np(precision, recall)
        
        # Create confusion matrix for compatibility
        cm = np.array([[tn, fp], [fn, tp]])
        
        if verbose:
            print(f"\n📊 Amamethrikhi Okuhlukanisa (Classification Metrics)")
            print(f"  Threshold: {threshold}")
            print(f"  Human threshold:  {human_thresh:.4f}")
            print(f"  Model threshold:  {model_thresh:.4f}")
            print(f"\n  Ukunemba (Accuracy):   {accuracy:.4f}")
            print(f"  Ukunembile (Precision): {precision:.4f}")
            print(f"  Ukukhumbula (Recall):   {recall:.4f}")
            print(f"  I-F1 Score:             {f1:.4f}")
            print(f"\n  Confusion Matrix:")
            print(f"                    Okubikezelwe (Predicted)")
            print(f"                  Akufani  Kuyafana")
            print(f"    Iqiniso   Akufani  {tn:4d}   {fp:4d}")
            print(f"    (Actual)  Kuyafana {fn:4d}   {tp:4d}")
            print(f"\n  Amaqiniso Apositivi (True Positives):  {tp}")
            print(f"  Amaqiniso Anegativi (True Negatives):  {tn}")
            print(f"  Amanga Apositivi (False Positives):     {fp}")
            print(f"  Amanga Anegativi (False Negatives):     {fn}")
        
        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'confusion_matrix': cm,
            'true_positives': int(tp),
            'true_negatives': int(tn),
            'false_positives': int(fp),
            'false_negatives': int(fn),
            'human_threshold': human_thresh,
            'model_threshold': model_thresh,
            'y_true': y_true,
            'y_pred': y_pred
        }
    
    def evaluate_similarity(self, word_pairs, compute_classification=True, 
                          threshold='median'):
        """
        Evaluate isiZulu model on word pairs with both correlation and classification metrics.
        
        Args:
            word_pairs: List of tuples (word1, word2, human_score)
            compute_classification: Whether to compute precision/recall/F1
            threshold: Threshold for binarization ('median', 'mean', or numeric)
            
        Returns:
            Dictionary with all evaluation results
        """
        model_scores = []
        human_scores = []
        missing_words = set()
        valid_pairs = []
        
        for word1, word2, human_score in word_pairs:
            sim = self.get_similarity(word1, word2)
            
            if sim is not None:
                model_scores.append(sim)
                human_scores.append(human_score)
                valid_pairs.append((word1, word2))
            else:
                missing_words.add(word1)
                missing_words.add(word2)
        
        # Compute correlations
        if len(model_scores) < 2:
            print("❌ Amapheyari awanele ukubala amamethrikhi (Not enough valid pairs)")
            return None
        
        spearman_corr, spearman_pval = spearmanr(human_scores, model_scores)
        pearson_corr, pearson_pval = pearsonr(human_scores, model_scores)
        
        coverage = len(model_scores) / len(word_pairs) * 100
        
        # Print correlation results
        print(f"\n📊 Amamethrikhi Wokuhlobana (Correlation Metrics):")
        print(f"  Ukumbozwa:    {len(model_scores)}/{len(word_pairs)} pairs ({coverage:.1f}%)")
        print(f"  Spearman ρ:   {spearman_corr:.4f} (p={spearman_pval:.4e})")
        print(f"  Pearson r:    {pearson_corr:.4f} (p={pearson_pval:.4e})")
        
        results = {
            'spearman': spearman_corr,
            'spearman_pval': spearman_pval,
            'pearson': pearson_corr,
            'pearson_pval': pearson_pval,
            'coverage': coverage,
            'n_pairs': len(model_scores),
            'total_pairs': len(word_pairs),
            'missing_words': missing_words,
            'valid_pairs': valid_pairs
        }
        
        # Always compute classification metrics
        classification_results = self.compute_classification_metrics(
            human_scores, model_scores, threshold=threshold, verbose=compute_classification
        )
        results.update(classification_results)
        if missing_words:
                    print(f"\n⚠️  Amagama angekho (Missing words): {len(missing_words)}")
                    if len(missing_words) <= 15:
                        print(f"  Amagama angekho: {', '.join(sorted(missing_words))}")
                
        return results
    
    def load_isizulu_similarity_dataset(self, filepath):
        """
        Load isiZulu word similarity dataset.
        
        Expected format (CSV or TSV):
        word1, word2, similarity_score
        
        Example:
        umfazi,indoda,5.2
        ingane,umntwana,9.5
        inja,ikati,6.8
        
        Returns:
            List of tuples (word1, word2, similarity_score)
        """
        word_pairs = []
        
        if not os.path.exists(filepath):
            print(f"⚠️  Ifayela ayitholakali (File not found): {filepath}")
            return None
        
        # Detect delimiter
        with open(filepath, 'r', encoding='utf-8') as f:
            first_line = f.readline()
            delimiter = '\t' if '\t' in first_line else ','
        
        with open(filepath, 'r', encoding='utf-8') as f:
            reader = csv.reader(f, delimiter=delimiter)
            
            # Check if there's a header
            first_row = next(reader)
            if not first_row[0].replace('.','').replace('-','').isdigit():
                # Has header, skip it
                pass
            else:
                # No header, process first row
                if len(first_row) >= 3:
                    word1 = first_row[0].strip().lower()
                    word2 = first_row[1].strip().lower()
                    score = float(first_row[2])
                    word_pairs.append((word1, word2, score))
            
            for row in reader:
                if len(row) >= 3:
                    word1 = row[0].strip().lower()
                    word2 = row[1].strip().lower()
                    try:
                        score = float(row[2])
                        word_pairs.append((word1, word2, score))
                    except ValueError:
                        continue
        
        print(f"✅ Kulayishwe amapheyari {len(word_pairs)} amagama (Loaded {len(word_pairs)} word pairs)")
        return word_pairs
    
    def load_simlex999(self, filepath='SimLex-999.txt'):
        """
        Load SimLex-999 dataset (English or translated to isiZulu).
        
        Format: word1, word2, POS, SimLex999, conc(w1), conc(w2), ...
        
        Returns:
            List of tuples (word1, word2, similarity_score)
        """
        word_pairs = []
        
        if not os.path.exists(filepath):
            print(f"⚠️  SimLex-999 ifayela ayitholakali: {filepath}")
            print("📥 Dawuniloda kusuka ku: https://fh295.github.io/simlex.html")
            return None
        
        with open(filepath, 'r', encoding='utf-8') as f:
            reader = csv.reader(f, delimiter='\t')
            next(reader)  # Skip header
            
            for row in reader:
                if len(row) >= 4:
                    word1 = row[0].strip().lower()
                    word2 = row[1].strip().lower()
                    score = float(row[3])
                    word_pairs.append((word1, word2, score))
        
        print(f"✅ Kulayishwe {len(word_pairs)} amapheyari amagama kusuka ku-SimLex-999")
        return word_pairs
    
    def load_wordsim353(self, filepath='wordsim353.csv'):
        """
        Load WordSim-353 dataset (English or translated to isiZulu).
        
        Format: Word 1, Word 2, Human (mean)
        
        Returns:
            List of tuples (word1, word2, similarity_score)
        """
        word_pairs = []
        
        if not os.path.exists(filepath):
            print(f"⚠️  WordSim-353 ifayela ayitholakali: {filepath}")
            return None
        
        with open(filepath, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader)  # Skip header
            
            for row in reader:
                if len(row) >= 3:
                    word1 = row[0].strip().lower()
                    word2 = row[1].strip().lower()
                    score = float(row[2])
                    word_pairs.append((word1, word2, score))
        
        print(f"✅ Kulayishwe {len(word_pairs)} amapheyari amagama kusuka ku-WordSim-353")
        return word_pairs
    
    def evaluate_isizulu_dataset(self, filepath, 
                                  compute_classification=True, 
                                  threshold='median'):
        """
        Evaluate model on isiZulu word similarity dataset.
        
        Returns:
            Dictionary with all evaluation results
        """
        print("\n" + "="*60)
        print("📊 Ukuhlolwa Kwedathasethe YesiZulu (isiZulu Dataset Evaluation)")
        print("="*60)
        
        word_pairs = self.load_isizulu_similarity_dataset(filepath)
        if word_pairs is None:
            return None
        
        return self.evaluate_similarity(word_pairs, compute_classification, threshold)
    
    def evaluate_with_multiple_thresholds(self, word_pairs, 
                                         thresholds=['median', 'mean', 0.5, 0.6, 0.7]):
        """
        Evaluate classification metrics with multiple threshold values.
        Helps find the optimal threshold for isiZulu word classification.
        
        Args:
            word_pairs: List of tuples (word1, word2, human_score)
            thresholds: List of threshold values to try
            
        Returns:
            Dictionary mapping thresholds to their metrics, and best threshold
        """
        # Get scores
        model_scores = []
        human_scores = []
        
        for word1, word2, human_score in word_pairs:
            sim = self.get_similarity(word1, word2)
            if sim is not None:
                model_scores.append(sim)
                human_scores.append(human_score)
        
        if len(model_scores) < 2:
            print("❌ Amapheyari awanele (Not enough valid pairs)")
            return None, None
        
        print("\n" + "="*70)
        print("🎯 UKUHLAZIYA I-THRESHOLD (THRESHOLD ANALYSIS)")
        print("="*70)
        
        results = {}
        best_f1 = 0
        best_threshold = None
        
        for thresh in thresholds:
            print(f"\n{'─'*70}")
            print(f"I-Threshold: {thresh}")
            print(f"{'─'*70}")
            
            metrics = self.compute_classification_metrics(
                human_scores, model_scores, threshold=thresh, verbose=True
            )
            results[str(thresh)] = metrics
            
            if metrics['f1_score'] > best_f1:
                best_f1 = metrics['f1_score']
                best_threshold = thresh
        
        print("\n" + "="*70)
        print(f"🏆 I-Threshold Engcono Kakhulu (Best Threshold): {best_threshold} (F1 = {best_f1:.4f})")
        print("="*70)
        
        return results, best_threshold
    
    def evaluate_all_benchmarks(self, isizulu_path=None,
                                simlex_path=None, 
                                wordsim_path=None,
                                compute_classification=True,
                                threshold='median'):
        """
        Evaluate isiZulu model on all available benchmarks.
        
        Args:
            isizulu_path: Path to isiZulu word similarity dataset
            simlex_path: Path to SimLex-999 (English or isiZulu translation)
            wordsim_path: Path to WordSim-353 (English or isiZulu translation)
            
        Returns:
            Dictionary with results for each benchmark
        """
        results = {}
        
        print("\n" + "="*70)
        print("🎯 UKUHLOLWA OKUPHELELE (COMPREHENSIVE BENCHMARK EVALUATION)")
        print("="*70)
        
        # isiZulu Dataset
        if isizulu_path:
            print("\n1️⃣  Idathasethe YesiZulu (isiZulu Dataset)")
            isizulu_results = self.evaluate_isizulu_dataset(
                isizulu_path, compute_classification, threshold
            )
            if isizulu_results:
                results['isizulu'] = isizulu_results
        
        # SimLex-999
        if simlex_path:
            print("\n2️⃣  SimLex-999")
            word_pairs = self.load_simlex999(simlex_path)
            if word_pairs:
                simlex_results = self.evaluate_similarity(
                    word_pairs, compute_classification, threshold
                )
                if simlex_results:
                    results['simlex999'] = simlex_results
        
        # WordSim-353
        if wordsim_path:
            print("\n3️⃣  WordSim-353")
            word_pairs = self.load_wordsim353(wordsim_path)
            if word_pairs:
                wordsim_results = self.evaluate_similarity(
                    word_pairs, compute_classification, threshold
                )
                if wordsim_results:
                    results['wordsim353'] = wordsim_results
        
        # Summary
        if results:
            self.print_summary(results)
        
        return results
    
    def print_summary(self, results):
        """Print a comprehensive summary of all results in English and isiZulu."""
        print("\n" + "="*70)
        print("📈 ISIFINYEZO ESIPHELELE (COMPREHENSIVE SUMMARY)")
        print("="*70)
        
        for benchmark, data in results.items():
            print(f"\n{benchmark.upper()}:")
            print(f"  {'─'*60}")
            print(f"  Amamethrikhi Wokuhlobana (Correlation Metrics):")
            print(f"    Spearman ρ:   {data['spearman']:.4f}")
            print(f"    Pearson r:    {data['pearson']:.4f}")
            print(f"    Ukumbozwa:    {data['coverage']:.1f}%")
            
            if 'accuracy' in data:
                print(f"\n  Amamethrikhi Okuhlukanisa (Classification Metrics):")
                print(f"    Ukunemba (Accuracy):    {data['accuracy']:.4f}")
                print(f"    Ukunembile (Precision): {data['precision']:.4f}")
                print(f"    Ukukhumbula (Recall):   {data['recall']:.4f}")
                print(f"    I-F1 Score:             {data['f1_score']:.4f}")


def create_sample_isizulu_dataset():
    """Create a sample isiZulu word similarity dataset for testing."""
    sample_data = """igama1,igama2,isilinganiso
umfazi,indoda,5.2
ingane,umntwana,9.5
inja,ikati,6.8
indlu,ikhaya,8.7
umfula,ulwandle,6.5
isikole,isikhungo,7.8
imali,uhulumeni,4.2
uthisha,umfundi,7.5
isitsha,indishi,9.2
ibhola,umdlalo,7.8
ukudla,ukuphuza,6.5
usuku,ubusuku,3.2
umuntu,ubuntu,8.5
itheku,idolobha,9.1
izulu,umoya,6.8"""
    
    with open('isizulu_similarity_sample.csv', 'w', encoding='utf-8') as f:
        f.write(sample_data)
    
    print("✅ Kwakhiwe i-isizulu_similarity_sample.csv (Created isiZulu sample dataset)")
    return 'isizulu_similarity_sample.csv'

# results = evaluator.evaluate_isizulu_dataset(
#     "my_real_isizulu_similarity_dataset.csv",
#     compute_classification=True,
#     threshold="median"
# )

# Demo usage
if __name__ == "__main__":
    print("="*70)
    print("📊 IsiZulu Word Similarity Benchmark Evaluator - FIXED")
    print("   Okuqukethwe: Precision, Recall, Accuracy, ne-F1 Score")
    print("="*70)
    
    print("\n✅ All syntax errors fixed!")
    print("✅ Helper functions included (confusion_matrix_np, accuracy_np, etc.)")
    print("✅ Ready to use with your isiZulu word embeddings!")
    
    print("\n" + "="*70)
    print("📖 QUICK START")
    print("="*70)
    
    print("""
from gensim.models import Word2Vec
from isizulu_benchmark_evaluator_fixed import IsiZuluBenchmarkEvaluator

# Load your isiZulu Word2Vec model
model = Word2Vec.load("isizulu_word2vec.bin")

# Create evaluator
evaluator = IsiZuluBenchmarkEvaluator(model=model)

# Evaluate on your isiZulu dataset
results = evaluator.evaluate_isizulu_dataset(
    'isizulu_word_similarity.csv',
    compute_classification=True,
    threshold='median'
)

# Print all metrics
          
print(f"Spearman ρ: {results['spearman']:.4f}")
print(f"Accuracy: {results['accuracy']:.4f}")
print(f"Precision: {results['precision']:.4f}")
print(f"Recall: {results['recall']:.4f}")
print(f"F1 Score: {results['f1_score']:.4f}")
""")
    
    print("\n" + "="*70)
    print("🧪 Creating sample isiZulu dataset for testing...")
    print("="*70)
    
    sample_file = create_sample_isizulu_dataset()
    print(f"\n✅ Sample file created: {sample_file}")
    print("\n🚀 Ready to evaluate your isiZulu word embeddings!")



    if __name__ == "__main__":
        print("="*70)
        print("📊 IsiZulu Word Similarity Benchmark Evaluator - DEMO RUN")
        print("="*70)

        # 1️⃣ Create sample dataset
        sample_file = create_sample_isizulu_dataset()

        # 2️⃣ Define Word2Vec hyperparameters (adjust these!)
        w2v_params = {
            "vector_size": 50,   # size of embedding vectors
            "window": 5,         # context window size
            "min_count": 1,      # minimum frequency to include word
            "epochs": 300,        # training epochs
            "sg": 1              # skip-gram (1) or CBOW (0)
        }

        print(f"\n🧪 Creating a dummy Word2Vec model with hyperparameters:\n{w2v_params}")

        # 3️⃣ Prepare sample sentences for training
        sentences = [
            ["umfazi", "indoda", "umntwana", "umfundi", "uthisha", "indlu", "ikhaya", "umfula",       "ulwandle"],
            ["inja", "ikati", "ibhola", "umdlalo", "ukudla", "ukuphuza", "usuku", "ubusuku", 
            "umuntu", "ubuntu", "itheku", "idolobha", "izulu", "umoya", "isitsha", "indishi", 
            "isikole", "isikhungo", "imali", "uhulumeni"],

           ["Ngidode", "ngani", "kulesi", "sikhathi", "esikuso", "lapho", "abobulili", "bami",
           "bephenduka", "izinqawunqawu", "ezijuqa", "obunye", "ubulili"],

            ["Uma", "kukhona", "othinta", "ibizo", "labesilisa", "esidlangalaleni",
     "sengiye", "ngixinwe", "yisimo", "sokushobinga", "esingekho", "kuhle",
     "komuntu", "onesichenene", "njengendlela", "yokubalekela", "ichilo",
     "lokuthi", "owesilisa", "uyini", "emehlweni", "ezwe", "namuhla"],

["Uma", "kuthintwa", "igama", "lobaba", "ngenganywa", "ukuchobozela",
     "okusezingeni", "lomlobokazi", "oqala", "ukukhuluma", "noyisezala"],

    ["Abaphike", "abaphikayo"],

    ["Vele", "sidla", "gqokweni", "lunye", "nabenze", "inkinga", "yesihlava",
     "sokubulawa", "abesifazane", "umnqakiswano", "wobulili"],

    ["Kodwa", "amanzi", "asengene", "endlini", "agelezela", "eziko"],

    ["Nezikhunyana", "zobuntu", "ebezisavutha", "seziyacima", "amanzi",
     "aseyokwengama", "umsamo"],

    ["Kubi"],

    ["Ambalwa", "amakhaya", "anesakhiwo", "sobuntu", "sekwakhiwa", "amathuna",
     "okungcwaba", "izidumbu", "ezikhiqizwa", "ngamadoda", "abulala",
     "omama", "bangomuso"],

    ["Ikhaya", "lakhiwa", "ubaba", "nomama", "uma", "kukhona", "abasocongwayo",
     "usho", "ukuthi", "into", "ekhaya", "izoba", "umlando", "esizukulwaneni",
     "esizayo"],

    ["Esikhathini", "esingengakanani", "akakho", "phakathi", "kwethu",
     "ozokwazi", "ukugxumela", "ethala", "ahlenge", "izihlangu", "namaklwa",
     "obabamkhulu", "abakha", "ngawo", "lesi", "sizwe"],

    ["Ukwembuleka", "kwethu", "izinqe", "sikubona", "ngokuthi", "kube",
     "namadoda", "akhalisa", "okwezimfengwane", "esola", "abesifazane",
     "ngokundinda"],

    ["Bandinda", "nobani"],

    ["Asikho", "yini", "amadoda", "endidiyeleni", "yokundinda"],

    ["Ukhona", "yini", "onempendulo", "kule", "mbedumehlwana", "yamadoda",
     "akhipha", "imiphefumulo", "yabesifazane"],

    ["Akunamkhankaso", "osebenzayo"],

    ["Uma", "kuke", "kwaba", "nenhlabamkhosi", "kuba", "sengathi", "kunamadoda",
     "ahilwa", "yidimoni", "lokuthi", "awenze", "mawala", "le", "nkilayitheka",
     "engachazeki"],

    ["Sekulukhuni", "nokubabaza", "lo", "mkhuhlane"],

    ["Isuke", "ingenwe", "hlobo", "luni", "lofufunyane", "indoda",
     "ehlahlela", "isimomondiya", "esithandayo"],

    ["Angisayiphathi-ke", "eyokumhlinza", "ebuka"],

    ["Uma", "umbulala", "kusho", "ukuthi", "isithandwa", "sakho", "usibona",
     "njengemfuyo", "okumele", "ihlatshwe"],

    ["Okubhekene", "nathi", "madoda", "ukuthi", "ayikho", "enye", "indawo",
     "lapho", "isixazululo", "siyophuma", "khona"],

    ["Kukuwo", "amadoda", "esizukulwane", "sethu", "ukuba", "asukume",
     "ngobuningi", "athi", "phinde"],

    ["Uma", "sikhuza", "mawala", "njengamadoda", "siluphehle", "uzwathi",
     "lokuthi", "ukuphakamisa", "isandla", "nesikhali", "kowesifazane",
     "kuyalulaza"],

    ["Iqiniso", "lithi", "le", "nhlekelele", "akuseyona", "eyamaphoyisa",
     "eyethu", "isizwe"],

    ["Mina", "ngikuthola", "kunzima", "ukuqhubeka", "ngibizwe", "ngendoda",
     "ezweni", "eseliphenduke", "isilaha"],

    ["Akuseyona", "indaba", "yokuntela", "le"],

    ["Ayikhulunywe", "le", "sizwe", "sakwethu"],

    ["Incithakalo", "isingaphezu", "kwamandla", "omuntu"],

    ["USergeant", "Mduduzi", "Buthelezi", "41", "utholakale", "eseshonile",
 "emgwaqweni", "uR603", "eMbumbulu", "ngemuva", "kokuthi", "imoto",
 "abehambe", "ngayo", "ishayisane", "mahlanze", "neloli"],

["Kepha", "okuxake", "umndeni", "wakhe", "wukuthi", "ubeshile",
 "ebusweni", "enenxeba", "lokudutshulwa", "esandleni"],

["Umndeni", "kaNksz", "Nompumelelo", "Mthembu", "36", "wakwaZ",
 "eMlaza", "osesesimweni", "esibucayi", "esibhedlela", "uthe",
 "kuyacaca", "ukuthi", "uButhelezi", "ubevele", "ehlose",
 "ukuzibulala", "ngokuziphosa", "elolini", "ngemuva",
 "kokushiya", "ezitshela", "ukuthi", "umbulele"],

 ["Uninakhulu", "kaNksz", "Mthembu", "ohlala", "nengane", "yalaba",
 "eneminyaka", "emihlanu", "uNkk", "Miriam", "Mthembu", "77",
 "uthe", "ubonga", "uMdali", "ngokusindisa", "umzukulu", "wakhe"],

["UMpume", "usinde", "ngomusa", "kaNkulunkulu"],

["Umkhwenyana", "ubethi", "uyaqeda", "ngaye"],

["Umdubule", "ezindaweni", "ezibucayi", "enhlafunweni",
 "phezu", "kweso", "nasesifubeni", "ngasenhliziyweni"],

["Ngokusho", "kwezingane", "zabo", "zabafana", "ezimbili",
 "ebezikhona", "kwenzeka", "lokhu"],

["Umkhwenyana", "uqale", "washayela", "ucingo", "lo", "omncane",
 "ona14", "wambuza", "ukuthi", "babuya", "nini", "ekhaya"],

["Umtshele", "ukuthi", "bazobuya", "ngakusasa", "ngeSonto",
 "okusolwa", "ukuthi", "yikho", "okumcasulile"],

["Ushaye", "imoto", "esuka", "kubo", "kwaD", "waqonda",
 "lapho", "bekhona"],

["UMpume", "ubehleli", "phansi", "edla", "iphalishi",
 "ngesikhathi", "umkhwenyana", "edubula", "ebhekise", "kuye"],

["Utshele", "izingane", "zakhe", "ukuthi", "aziphume"],

["Lo", "omncane", "wabaleka", "wayobiza", "omakhelwane"],

["Lo", "omdala", "ona18", "unqabile", "ukuphuma",
 "wathi", "uzofa", "lapho", "kufa", "khona", "unina"],

["UMpume", "usethele", "umkhwenyana", "ngephalishi",
 "wathola", "ithuba", "lokubaleka", "angene", "ethoyilethi"],

["Umlandela", "waze", "wafika", "khona", "wamdubula"],

["Babangisana", "ngesibhamu", "sebelele", "phansi"],

["Umkhwenyana", "wadubuleka", "engalweni",
 "isibhamu", "sathathwa", "yindodana", "yabo", "endala"],

["Wacela", "ukuthi", "amnikeze", "isibhamu",
 "yanqaba", "wabe", "esephuma", "ehamba"],

["USipho", "Mthembu", "82", "uthe", "bathole", "ucingo",
 "lubazisa", "ukuthi", "umzukulu", "wabo",
 "usedutshulwe", "nguyise", "wezingane", "zakhe"],

["Bathole", "umbiko", "wokuthi", "umkhwenyana",
 "utholwe", "eseshonile", "eMbumbulu",
 "ngemuva", "kokuziphosa", "elolini"],

["Kusho", "ukuthi", "ubevele", "ehlele",
 "ukuzibulala"],

["Wathatha", "isibhamu", "somsebenzi",
 "wayohlasela", "ngaso"],

["Sidumele", "kakhulu"],

["Izingane", "zizoyiswa", "kolulekwa",
 "ngokwengqondo"],

["AMAVA", "eminyaka", "ewu-25", "ewuthisha",
 "ahlwanyele", "imbewu", "yomqondo",
 "webhizinisi", "eliwusizo", "ezinganeni",
 "kuNkk", "Noxolo", "Nokwe"],

["Ohlangane", "nendodakazi", "yakhe",
 "uNkk", "Lelethu", "Hlongwa",
 "-", "bavula", "isikole",
 "esiyinkulisa", "iFuture", "Harvest",
 "esiwusizo", "lwsesisekelo",
 "semfundo", "ezinganeni"],

["UNkk", "Nokwe", "ozinze", "ePinetown",
 "ngaleli", "bhizinisi", "aliqale",
 "ngo-2017", "uthe", "wayesefuna",
 "inselelo", "entsha", "ngenkathi",
 "enquma", "ukwesula", "emsebenzini",
 "wakhe", "wokuba", "wuthisha"],

["Kodwa", "yasala", "insila",
 "yokuthanda", "ukufundisa",
 "nokucija", "izingane"],

["Yilapho", "abona", "khona",
 "ukuthi", "isekhona", "indlela",
 "angingasebenzisa", "ubizo",
 "lwami", "ngayo"],

["Ukuze", "ngiqhubeke",
 "ngiqinisekise", "ukuthi",
 "izingane", "zikhula",
 "zinekusasa", "eliqhakazile"],

["Ngaleli", "bhizinisi",
 "eliyisisekelo", "semfundo",
 "kodado", "ngiqinisekisa",
 "ukuthi", "bakhula",
 "bekhaliphile"],

["Kusho", "yena"],

["Lokho", "sikwenzwa", "ngokuthi",
 "zonke", "izingane", "ezincane",
 "ezibhalisa", "esikoleni",
 "sethu", "zithole",
 "ulwazi", "olumqoka",
 "lokufunda", "ngokuqonda"],

["Bese", "futhi", "sigxila",
 "kakhulu", "ekuzicijeni",
 "ngesiNgisi", "ukusiza",
 "ukuthi", "kube", "lula",
 "uma", "seziqhubeka",
 "nemfundo"],

["Ngoba", "zizobe", "sezikwazi",
 "ukukhuluma", "ukufunda",
 "nokunye", "okweyamene",
 "nakho", "okuzokwenza",
 "zihluke", "kwezinye"],

["Kusho", "uNkk", "Nokwe"],

["Eqhuba", "uthe",
 "okunye", "okumphushe",
 "ukuqala", "iFuture",
 "Harvest"],

["Wukuthi", "esewuthisha",
 "oqashiwe", "wayeqaphela",
 "ukuthi", "izingane",
 "eziningi", "ziba",
 "nenkinga",
 "yokungakwazi",
 "ukufunda",
 "ngokuqonda"],

["Ezinye", "zihluleka",
 "wukukhuluma",
 "-", "okukhomba",
 "ukuthi", "kusuke",
 "kukhona", "okushodile",
 "noma", "kwangahamba",
 "kahle"],

["Endleleni",
 "ezifundiswe",
 "ngayo",
 "zisencane"]


        ]

        # 4️⃣ Train Word2Vec model
        dummy_model = Word2Vec(
            sentences,
            vector_size=w2v_params["vector_size"],
            window=w2v_params["window"],
            min_count=w2v_params["min_count"],
            sg=w2v_params["sg"]
        )
        dummy_model.train(sentences, total_examples=len(sentences), epochs=w2v_params["epochs"])
        print("✅ Dummy Word2Vec model created.\n")

        # 5️⃣ Initialize evaluator
        evaluator = IsiZuluBenchmarkEvaluator(model=dummy_model)

        # 6️⃣ Evaluate sample dataset
        print("📊 Evaluating sample isiZulu dataset...")
        results = evaluator.evaluate_isizulu_dataset(sample_file, compute_classification=True, threshold='median')

        # 7️⃣ Print a final summary
        if results:
            evaluator.print_summary({'Sample Dataset': results})

        print("\n🚀 Demo evaluation completed!")


📊 IsiZulu Word Similarity Benchmark Evaluator - FIXED
   Okuqukethwe: Precision, Recall, Accuracy, ne-F1 Score

✅ All syntax errors fixed!
✅ Helper functions included (confusion_matrix_np, accuracy_np, etc.)
✅ Ready to use with your isiZulu word embeddings!

📖 QUICK START

from gensim.models import Word2Vec
from isizulu_benchmark_evaluator_fixed import IsiZuluBenchmarkEvaluator

# Load your isiZulu Word2Vec model
model = Word2Vec.load("isizulu_word2vec.bin")

# Create evaluator
evaluator = IsiZuluBenchmarkEvaluator(model=model)

# Evaluate on your isiZulu dataset
results = evaluator.evaluate_isizulu_dataset(
    'isizulu_word_similarity.csv',
    compute_classification=True,
    threshold='median'
)

# Print all metrics

print(f"Spearman ρ: {results['spearman']:.4f}")
print(f"Accuracy: {results['accuracy']:.4f}")
print(f"Precision: {results['precision']:.4f}")
print(f"Recall: {results['recall']:.4f}")
print(f"F1 Score: {results['f1_score']:.4f}")


🧪 Creating sample isiZulu dataset for